In [1]:
from coinbase.wallet.client import Client 
import requests
import time 
import os
from dotenv import load_dotenv
import psycopg2
from datetime import datetime 
from email.utils import parsedate_to_datetime
import pytz


In [2]:
load_dotenv('./.env')


True

In [3]:
api_key = os.getenv("api_key")
api_secret = os.getenv("api_secret")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")


In [4]:
#creating connection to railway postgres db: 
connection = psycopg2.connect(
        host = db_host, 
        database = db_name, 
        user = db_user, 
        password = db_password, 
        port = db_port
    )

    #initializing cursor
cursor = connection.cursor() 

In [5]:
cursor.execute("SELECT 1;")

In [6]:
result = cursor.fetchone()

In [7]:
print(result) # connection is active 

(1,)


In [8]:
#establishing connection to coinbase API
client = Client(api_key, api_secret)

#pulling price data for ethereum and bitcoin 
price_eth = client.get_spot_price(currency_pair= "ETH-USD")
price_btc = client.get_spot_price(currency_pair= "BTC-USD")

#adding timestamp to dictionaries 

#price_eth["datetime"] = datetime.now() 
#price_btc["datetime"] = datetime.now() 
 


In [9]:
price_btc

<APIObject @ 0x7fdf80d9f9c0> {
  "amount": "35082.96",
  "base": "BTC",
  "currency": "USD"
}

In [21]:
btc_dict = {"datetime": datetime.now(), 
            "base": price_btc["base"],
            "amount": price_btc["amount"], 
            "currency": price_btc["currency"]}

eth_dict = {"datetime": datetime.now(), 
            "base": price_eth["base"],
            "amount": price_eth["amount"], 
            "currency": price_eth["currency"]}
            

In [22]:
eth_dict

{'datetime': datetime.datetime(2023, 11, 6, 15, 25, 34, 303327),
 'base': 'ETH',
 'amount': '1906.455',
 'currency': 'USD'}

In [13]:
table_schema_btc = ("""
                    CREATE TABLE IF NOT EXISTS bitcoin
                    (id serial PRIMARY KEY, 
                     datetime timestamp, 
                     base char(3), 
                     amount numeric(10,4),
                     currency char(3));
                    """)

In [14]:
cursor.execute(table_schema_btc)

In [63]:
connection.commit()

In [15]:
insert_query = """ INSERT INTO bitcoin (datetime, base, amount, currency)
VALUES (%(datetime)s, %(base)s, %(amount)s, %(currency)s);
"""

In [16]:
cursor.execute(insert_query, btc_dict)

In [17]:
connection.commit()

In [23]:
#now trying ethereum table and insert statement 
table_schema_eth = ("""
                    CREATE TABLE IF NOT EXISTS ethereum
                    (id serial PRIMARY KEY, 
                     datetime timestamp, 
                     base char(3), 
                     amount numeric(10,4),
                     currency char(3));
                    """)

cursor.execute(table_schema_eth)

insert_query_eth = """ INSERT INTO ethereum (datetime, base, amount, currency)
VALUES (%(datetime)s, %(base)s, %(amount)s, %(currency)s);
"""

cursor.execute(insert_query_eth, eth_dict)

connection.commit() 


ProgrammingError: no results to fetch